In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

<h1>Basic data cleaning

In [ ]:
import numpy as np

In [ ]:
market_train_df['close_to_open'] =  np.abs(market_train_df['close'] / market_train_df['open'])

In [ ]:
market_train_df['assetName_mean_open'] = market_train_df.groupby('assetName')['open'].transform('mean')
market_train_df['assetName_mean_close'] = market_train_df.groupby('assetName')['close'].transform('mean')

# if open price is too far from mean open price for this company, replace it. Otherwise replace close price.
for i, row in market_train_df.loc[market_train_df['close_to_open'] >= 2].iterrows():
    if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
        market_train_df.iloc[i,5] = row['assetName_mean_open']
    else:
        market_train_df.iloc[i,4] = row['assetName_mean_close']
        
for i, row in market_train_df.loc[market_train_df['close_to_open'] <= 0.5].iterrows():
    if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
        market_train_df.iloc[i,5] = row['assetName_mean_open']
    else:
        market_train_df.iloc[i,4] = row['assetName_mean_close']

In [ ]:
market_train_df = market_train_df.loc[market_train_df['time'] >= '2010-01-01 22:00:00+0000']

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
bottom, top = market_train_df.returnsOpenNextMktres10.quantile(0.001), market_train_df.returnsOpenNextMktres10.quantile(0.999)
returns, binwidth = market_train_df.returnsOpenNextMktres10.clip(bottom, top), 0.005
plt.figure(figsize=(15,10))
plt.hist(returns,  bins=np.arange(min(returns), max(returns) + binwidth, binwidth))

In [ ]:
market_train_df.returnsOpenNextMktres10 = market_train_df.returnsOpenNextMktres10.clip(bottom, top)

<h1>Test-Validation Split

In [ ]:
X, Y = market_train_df.iloc[:, (market_train_df.columns != 'assetCode') & (market_train_df.columns != 'assetName') &(market_train_df.columns != 'time') & (market_train_df.columns != 'returnsOpenNextMktres10')], market_train_df['returnsOpenNextMktres10']

In [ ]:
split = int(len(X) * 0.8)
test_train_distsance = 20000
X_train, X_val = X[:split - test_train_distsance], X[split:]
Y_train, Y_val = Y[:split - test_train_distsance], Y[split:]

In [ ]:
print(len(X_val), len(Y_val))
universe_filter = market_train_df['universe'][split:] == 1.0
X_val = X_val[universe_filter]
Y_val = Y_val[universe_filter]
print(len(X_val), len(Y_val))

In [ ]:
# this is a time_val series used to calc the sigma_score later, applied split and universe filter
time_val = market_train_df['time'][split:][universe_filter]
assert len(time_val) == len(X_val)
time_train = market_train_df['time'][:split - test_train_distsance]
assert len(time_train) == len(X_train)

<h1>Metric Definition

In [ ]:
def sigma_score(preds, valid_data):
    df_time = valid_data.params['extra_time'] # will be injected afterwards
    labels = valid_data.get_label()
    
#    assert len(labels) == len(df_time)

    x_t = preds * labels #  * df_valid['universe'] -> Here we take out the 'universe' term because we already keep only those equals to 1.
    
    # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
    # is a pd.Series and call `group_by`
    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()

    return 'sigma_score', score, True

<h1>Fit basic LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
train_cols = X.columns.tolist()

lgb_train = lgb.Dataset(X_train.values, Y_train, feature_name=train_cols, free_raw_data=False)
lgb_val = lgb.Dataset(X_val.values, Y_val, feature_name=train_cols, free_raw_data=False)

lgb_train.params = {
    'extra_time' : time_train.factorize()[0]
}
lgb_val.params = {
    'extra_time' : time_val.factorize()[0]
}


In [ ]:
lgb_params = dict(
    objective = 'regression_l1',
    learning_rate = 0.1,
    num_leaves = 127,
    max_depth = -1,
#     min_data_in_leaf = 1000,
#     min_sum_hessian_in_leaf = 10,
    bagging_fraction = 0.75,
    bagging_freq = 2,
    feature_fraction = 0.5,
    lambda_l1 = 0.0,
    lambda_l2 = 1.0,
    metric = 'None', # This will ignore the loss objetive and use sigma_score instead,
    seed = 42 # Change for better luck! :)
)

In [ ]:
training_results = {}
model = lgb.train(lgb_params, lgb_train, num_boost_round=1000, valid_sets=(lgb_val,lgb_train), valid_names=('valid','train'), verbose_eval=25,
              early_stopping_rounds=100, feval=sigma_score, evals_result=training_results)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(training_results['valid']['sigma_score'])
plt.plot(training_results['train']['sigma_score'])

In [ ]:
features = market_train_df.columns[(market_train_df.columns != 'assetCode') & (market_train_df.columns != 'assetName') &(market_train_df.columns != 'time') & (market_train_df.columns != 'returnsOpenNextMktres10')]
print(features[0], features[1], features[13], features[7], features[9])

In [ ]:
x=lgb.plot_importance(model)
x.figure.set_size_inches(10, 30) 

In [ ]:
# You can only iterate through a result from `get_prediction_days()` once
# so be careful not to lose it once you start iterating.
days = env.get_prediction_days()

## Main Loop
Let's loop through all the days and make our random predictions.  The `days` generator (returned from `get_prediction_days`) will simply stop returning values once you've reached the end.

In [ ]:
def prepare_predictions(market_obs_df):
    market_obs_df['close_to_open'] =  np.abs(market_obs_df['close'] / market_obs_df['open'])
    market_obs_df['universe'] =  1
    market_obs_df['assetName_mean_open'] = market_obs_df.groupby('assetName')['open'].transform('mean')
    market_obs_df['assetName_mean_close'] = market_obs_df.groupby('assetName')['close'].transform('mean')

    # if open price is too far from mean open price for this company, replace it. Otherwise replace close price.
    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] >= 2].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']

    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] <= 0.5].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    prepare_predictions(market_obs_df)
    predictions = market_obs_df.iloc[:, (market_obs_df.columns != 'assetCode') & (market_obs_df.columns != 'assetName') &(market_obs_df.columns != 'time') ].values
    predictions_template_df.confidenceValue = model.predict(predictions).clip(-1, 1)
    env.predict(predictions_template_df)
print('Done!')

## **`write_submission_file`** function

Writes your predictions to a CSV file (`submission.csv`) in the current working directory.

In [ ]:
env.write_submission_file()

In [ ]:
# We've got a submission file!
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])

As indicated by the helper message, calling `write_submission_file` on its own does **not** make a submission to the competition.  It merely tells the module to write the `submission.csv` file as part of the Kernel's output.  To make a submission to the competition, you'll have to **Commit** your Kernel and find the generated `submission.csv` file in that Kernel Version's Output tab (note this is _outside_ of the Kernel Editor), then click "Submit to Competition".  When we re-run your Kernel during Stage Two, we will run the Kernel Version (generated when you hit "Commit") linked to your chosen Submission.

## Restart the Kernel to run your code again
In order to combat cheating, you are only allowed to call `make_env` or iterate through `get_prediction_days` once per Kernel run.  However, while you're iterating on your model it's reasonable to try something out, change the model a bit, and try it again.  Unfortunately, if you try to simply re-run the code, or even refresh the browser page, you'll still be running on the same Kernel execution session you had been running before, and the `twosigmanews` module will still throw errors.  To get around this, you need to explicitly restart your Kernel execution session, which you can do by pressing the Restart button in the Kernel Editor's bottom Console tab:
![Restart button](https://i.imgur.com/hudu8jF.png)